### regression_forecasting_and_predicting_practical_machine_learning_tutorial_with_python_p_5.mp4 
 The following notes is for a video by sendtex
 https://www.youtube.com/watch?v=QLVMqwpOLPk

In [31]:
from sklearn import preprocessing, cross_validation,svm
import math
from sklearn.linear_model import LinearRegression

In [32]:
import numpy as np

In [33]:
import quandl
#Video is asking to use Quandl -
# Note that Quandl does not work ( with a capital Q)
# quandl works just fine.. 
#
#

In [34]:
import pandas as pd

In [35]:
df=quandl.get('WIKI/GOOGL')

In [36]:
print(df.head())
# as we can see from the output - there are a lot of features.
# what we need is meaningful features.
#

              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2004-08-19  100.01  104.06   95.96  100.335  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.310  22834300.0          0.0   
2004-08-23  110.76  113.48  109.05  109.400  18256100.0          0.0   
2004-08-24  111.24  111.60  103.57  104.870  15247300.0          0.0   
2004-08-25  104.76  108.00  103.88  106.000   9188600.0          0.0   

            Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  \
Date                                                                   
2004-08-19          1.0  50.159839  52.191109  48.128568   50.322842   
2004-08-20          1.0  50.661387  54.708881  50.405597   54.322689   
2004-08-23          1.0  55.551482  56.915693  54.693835   54.869377   
2004-08-24          1.0  55.792225  55.972783  51.945350   52.597363   
2004-08-25          1.0  52.542193  54.167209  52.100830   53.1

In [37]:
#We remove columns we do NOT need in the data frame.. 
df=df[['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume']]

In [38]:
df.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0


In [39]:
# Here we create a NEW COLUMN "HL_PCT"  in the dataframe. 
# 

df['HL_PCT']=(df['Adj. High'] - df['Adj. Close'] ) / df['Adj. Close'] * 100


In [40]:
# Here we create a NEW COLUMN "PCT_Change"  in the dataframe. 
# 
df['PCT_Change']=(df['Adj. Close'] - df['Adj. Open'] ) / df['Adj. Open'] * 100

In [41]:
#df.columns
#We remove columns we do NOT need in the data frame.. 
df=df[['Adj. Close','HL_PCT','PCT_Change','Adj. Volume']]


In [42]:
df.head()

,Adj. Close,HL_PCT,PCT_Change,Adj. Volume
Date,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0
2004-08-20,54.322689,0.710922,7.227007,22834300.0
2004-08-23,54.869377,3.729433,-1.227880,18256100.0
2004-08-24,52.597363,6.417469,-5.726357,15247300.0
2004-08-25,53.164113,1.886792,1.183658,9188600.0


In [43]:
# Would the 'Adj. Close' column be a FEATURE for a lable ?
#It would have been lable if we were NOT planning on using HL_PCT
#Why?
# in HL_PCT feature, 'Adj. Close' has already been factored in.
#  Discussion in regression_features_and_labels_practical_machine_learning_tutorial_with_python_p_3.mp4
#   Not clear.. ( not understood at 01:30)
#

In [44]:
#We need to decide what is it that we want to forecast?
# Looking at the data, we would be interested, if we can forecase the closing price.
# This is an important step - Raj.
#

forecast_col = 'Adj. Close'
df.fillna(-9999,inplace=True) 

In [45]:
# How many values do we need to forecast out.
#  This is typically given as a percentage of the data frame.
# This can aswell be an hard coded number - Raj.
#
forecast_out = int(math.ceil(0.01*len(df)))
# We want to predict 1 percent of the data frame
#  that is we want to extend the data frame (?) by 1 percent.. 
#
forecast_out

32

In [46]:
# So far, we dealt with features.
# Now, we come across 'label' -
# When you come across 'label' think supervised learning.
#  Think dependent variable.
# This step creates a new column called label.
# The value in the lable column are notihng but the closing price of the stock
#  n days into the future..  ( where n = forecast_out)
#  Good explanation of shift is present here.. 
#    http://stackoverflow.com/questions/20095673/python-shift-column-in-pandas-dataframe-up-by-one
#  The side effect of this shift operation: 
#   The last n rows will have a value of NaN under the 'label' column.
#   Note that n = forecare_out
# We will eventually do a cleanup by calling dropna method. - look out for this.
#
df['label']=df[forecast_col].shift(-forecast_out)

In [47]:
df['label'].head()

Date
2004-08-19    69.399229
2004-08-20    68.752232
2004-08-23    69.639972
2004-08-24    69.078238
2004-08-25    67.839414
Name: label, dtype: float64

In [52]:
df.head()

,Adj. Close,HL_PCT,PCT_Change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0,69.399229
2004-08-20,54.322689,0.710922,7.227007,22834300.0,68.752232
2004-08-23,54.869377,3.729433,-1.227880,18256100.0,69.639972
2004-08-24,52.597363,6.417469,-5.726357,15247300.0,69.078238
2004-08-25,53.164113,1.886792,1.183658,9188600.0,67.839414


In [60]:
df.tail(10)

,Adj. Close,HL_PCT,PCT_Change,Adj. Volume,label
Date,,,,,
2016-12-02,764.46,0.790100,0.336002,1686840.0,NaN
2016-12-05,778.22,0.228727,1.067532,1680524.0,NaN
2016-12-06,776.18,1.172408,-0.513977,1694701.0,NaN
2016-12-07,791.47,0.066964,1.477018,2014582.0,NaN
2016-12-08,795.17,0.481658,0.279967,1603167.0,NaN
2016-12-09,809.45,0.061770,1.269861,1889430.0,NaN
2016-12-12,807.90,0.427033,0.382694,1620757.0,NaN
2016-12-13,815.34,1.098928,0.363126,2097969.0,NaN
2016-12-14,817.89,0.778833,0.241445,1744276.0,NaN


In [48]:
X=np.array(df.drop(['label'],1)) 
# The above command will keep everything in df 
# the df.drop command will drop the 'label' column from the data frame 
#  and return the rest of the columns -
#       	Adj. Close
#           HL_PCT
#           PCT_Change
#           Adj. Volume
# Note the 1 in the df.drop - what is it?
#    It is known as the axis. 0 stands for row slicing and 1 for column slicing.. 
#
# This table of 4 columns and N rows is retruned to the np.array method.
# np.array converts the table into an array and stores it in X
# 
#  Note that end of this step, df remains un-altered . Why?
#   We did NOT place df in the lvalue NOR did we use the inplace parameter for df.drop
#  
# Summary of this step is we collect all columns except the label column into X
#

In [49]:
X[0]

array([  5.03228418e+01,   3.71256291e+00,   3.24967503e-01,
         4.46590000e+07])

In [57]:
df.tail()

,Adj. Close,HL_PCT,PCT_Change,Adj. Volume,label
Date,,,,,
2016-12-09,809.45,0.061770,1.269861,1889430.0,NaN
2016-12-12,807.90,0.427033,0.382694,1620757.0,NaN
2016-12-13,815.34,1.098928,0.363126,2097969.0,NaN
2016-12-14,817.89,0.778833,0.241445,1744276.0,NaN
2016-12-15,815.65,0.901122,-0.209210,1696212.0,NaN


In [ ]:
X=preprocessing.scale(X)
# Scaling happens at this step. 
# This is also an important step. 
# In the data, there can be very large number and very small numbers. 
# This will wreak havoc on the algorithms. 
#  So, we can compute standardized scores for each data item. 
#  When we do this, we are said to have done 'scaling'
# Standardized scores (or z scores) is a basic statistic concept.
# After 'standardizing' the data, the inherent nature of the data as a whole does NOT change.
# So, it will still be useful for predicting.
# Some reading material is here:
#   http://scikit-learn.org/stable/modules/preprocessing.html
# Good reading material here: https://www.analyticsvidhya.com/blog/2016/07/practical-guide-data-preprocessing-python-scikit-learn/
# Some more reading material here: http://sebastianraschka.com/Articles/2014_about_feature_scaling.html
# and http://web.cs.wpi.edu/~ruiz/KDDRG/Resources/Python/


In [53]:
print(len(X))
X=X[:-forecast_out]
print(len(X))
X[0]
# See that somethere at the top, we had to do a shift operation on data frame. 
# After doing it, we copied data to X
# What this has done it, it have moved the label column up by 'forecast_out' rows. 
#  This left NaN values in label column (and in X) at the very bottom.
# SO, what this step does is effectively clear those last 'forecast_out' rows in array.
#

3105
3073


array([  5.03228418e+01,   3.71256291e+00,   3.24967503e-01,
         4.46590000e+07])

In [54]:
X_lately=X[-forecast_out:]
print(len(X_lately))
print(len(X))
# Whats happening here?
#  Array slicing has been used here. 
#  First line extracts the the last forecast_out elements of X into X_lately.
#  Remember that there are the elements which have NaN for the 'label' column. ?? Verify this.. 
#
#   For more on Array slicing - use the notes in RoughNotes.ipynb
#

32
3073


array([  7.97970000e+02,   1.31583894e-01,  -1.31410978e-01,
         2.13057100e+06])

In [59]:
X_lately[30]
# This will NOT match with what we see in df.tail() --- why?
# Note that we scaled the array X  and then created the X_lately.
# So, whats in X and X_lately is the standardized or Normalized or preprocessed data.
#


array([  8.19560000e+02,   2.37200449e+00,  -1.25069282e+00,
         4.35488400e+06])

In [ ]:
df.dropna(inplace=True)
y=np.array(df['label'])

# Here we are cleaning up df - are NaN values are cleared ( rows will be deleted) .
# Note that before cleanup we have NaN values 
#  in the last forecast_out rows under the "lable" column
# After doing this clean up of df, we populate the label values into the array y
#


In [ ]:
y[0]

In [ ]:
X_train,X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.2)
# We have only one data set to work with. 
# How do we create a train dataset and a test dataset?
# Not a problem, we do a manual split of the array X and y
# or use the cross_validation packages train_test_split function.
# In this example, we have specified that the test_size be 0.2
#   what this means is we want to create a test dataset with 
#         20% of the data from main dataset.
# Train dataset will be the complement of test dataset - that is 80% in this case.
#
# End of the statement, we would have the following datasets:
# Training datasets X_train and y_train
# Test Datasets:    X_test and y_test
#


In [ ]:
clf = LinearRegression(n_jobs=-1)
# n_jobs = -1 is telling the program this:
#   this is a parameter about threading.
#   When this program goes about finding an optimal value - this can be done in threads.
#    when we give  a value of 2 for n_jobs, we ask that to run it in two threads
#    when we give a value of -1 , what we are saying it - we are leaving it to the program
#     to find the best number of threads given the limitation of the hardware and software 
#     the program is running under!
# What about the LinearRegression() function call - 
#  that function calls returns an object of type linear_model.
# We can use methods in that class(object) for fitting the data and do other tasks.
# Whats happening in the next line is we are fitting the data:
clf.fit(X_train,y_train)

In [ ]:
accuracy = clf.score(X_test,y_test)
# or accuracy = clf.score(X_test,y_test)
accuracy
# what we see here is the squared error


R-squared is a statistical measure of how close the data are to the fitted regression line. It is also known as the coefficient of determination, or the coefficient of multiple determination for multiple regression. 0% indicates that the model explains none of the variability of the response data around its mean.

In [ ]:
# The accuracy we got is 96% accuracy on predicting what the price would be 30 days into
#  future 
# 30 days ? remember the forecast_out variable.
#


In [ ]:
forecast_set = clf.predict(X_lately)
print(forecast_set,accuracy, forecast_out)
#Here we are predicting data.
